# Lets get dis money

add ur data to the data folder and add any things u have been doing here

In [10]:
import pandas as pd
import numpy as np
import sklearn as sk
import scipy as sp

In [11]:
qb_training_data = pd.read_csv("../data/qb_draft_data/quarterback_training.csv")

In [12]:
qb_training_data

,DraftYear,Round,Pick,Age,GamesPlayed,Completions,Attempts,Yards,Touchdowns,Interceptions,RushAttempts,RushYards,RushTouchdowns,Player,College,Conference,Team,Heisman,Verdict
0,2013,1,16,23,43,600,897,7741,47,28,298,827,11,EJ Manuel,Florida St.,Atlantic Coast,BUF,0,0
1,2013,2,39,22,44,988,1465,11662,98,21,245,342,4,Geno Smith,West Virginia,Big 12,NYJ,0,0
2,2013,3,73,23,36,646,1069,7411,63,31,111,-281,3,Mike Glennon,North Carolina St.,Atlantic Coast,TAM,0,0
3,2013,4,98,23,47,1001,1562,12327,116,48,132,-113,6,Matt Barkley,USC,Pac-12,PHI,0,0
4,2013,4,110,23,47,791,1312,9190,70,28,242,168,5,Ryan Nassib,Syracuse,Atlantic Coast,NYG,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,1999,4,131,23,34,357,651,5118,33,24,199,547,14,Aaron Brooks,Virginia,Atlantic Coast,GNB,0,1
124,1999,7,227,23,23,244,480,4401,36,13,324,1314,23,Michael Bishop,Kansas St.,Big 12,NWE,0,0
125,1999,7,245,23,35,281,497,3862,26,15,80,-245,2,Scott Covington,Miami (FL),Atlantic Coast,CIN,0,0
126,1998,1,1,22,45,863,1381,11201,89,33,153,-181,12,Peyton Manning,Tennessee,Southeastern,IND,0,1


one way i think we can approach this is classification where the label is round? it looks like the data above doesnt have enough for each overall pick to make that a label. so ig for now we can roll w round and when we find a bigger one we can change to overall pick. it shouldnt be too much work to make that switch as in the end the data that we classify based on should be the same.

In [13]:
# Creating label array:
qb_training_labels = qb_training_data["Round"]
print(qb_training_labels)

0      1
1      2
2      3
3      4
4      4
      ..
123    4
124    7
125    7
126    1
127    1
Name: Round, Length: 128, dtype: int64


In [14]:
# also lets drop some unnecessary things from the data
qb_training_data = qb_training_data.drop(columns = ["Round", "Pick", "DraftYear", "Player", "Team", "Verdict"])
qb_training_data

,Age,GamesPlayed,Completions,Attempts,Yards,Touchdowns,Interceptions,RushAttempts,RushYards,RushTouchdowns,College,Conference,Heisman
0,23,43,600,897,7741,47,28,298,827,11,Florida St.,Atlantic Coast,0
1,22,44,988,1465,11662,98,21,245,342,4,West Virginia,Big 12,0
2,23,36,646,1069,7411,63,31,111,-281,3,North Carolina St.,Atlantic Coast,0
3,23,47,1001,1562,12327,116,48,132,-113,6,USC,Pac-12,0
4,23,47,791,1312,9190,70,28,242,168,5,Syracuse,Atlantic Coast,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,23,34,357,651,5118,33,24,199,547,14,Virginia,Atlantic Coast,0
124,23,23,244,480,4401,36,13,324,1314,23,Kansas St.,Big 12,0
125,23,35,281,497,3862,26,15,80,-245,2,Miami (FL),Atlantic Coast,0
126,22,45,863,1381,11201,89,33,153,-181,12,Tennessee,Southeastern,0


## Feature Design

The first thing that comes to mind is average yards per pass/rush attempt.

In [15]:
qb_training_data["Passing Yard per attempt"] = qb_training_data["Yards"] / qb_training_data["Attempts"]
qb_training_data["Rushing Yard per attempt"] = qb_training_data["RushYards"] / qb_training_data["RushAttempts"]
qb_training_data = qb_training_data.drop(columns = ["Yards", "RushYards", "RushAttempts"])
qb_training_data

,Age,GamesPlayed,Completions,Attempts,Touchdowns,Interceptions,RushTouchdowns,College,Conference,Heisman,Passing Yard per attempt,Rushing Yard per attempt
0,23,43,600,897,47,28,11,Florida St.,Atlantic Coast,0,8.629877,2.775168
1,22,44,988,1465,98,21,4,West Virginia,Big 12,0,7.960410,1.395918
2,23,36,646,1069,63,31,3,North Carolina St.,Atlantic Coast,0,6.932647,-2.531532
3,23,47,1001,1562,116,48,6,USC,Pac-12,0,7.891805,-0.856061
4,23,47,791,1312,70,28,5,Syracuse,Atlantic Coast,0,7.004573,0.694215
...,...,...,...,...,...,...,...,...,...,...,...,...
123,23,34,357,651,33,24,14,Virginia,Atlantic Coast,0,7.861751,2.748744
124,23,23,244,480,36,13,23,Kansas St.,Big 12,0,9.168750,4.055556
125,23,35,281,497,26,15,2,Miami (FL),Atlantic Coast,0,7.770624,-3.062500
126,22,45,863,1381,89,33,12,Tennessee,Southeastern,0,8.110789,-1.183007


Next, we can normalize the number of passing and rushing touchdowns by the number of games played.

In [16]:
qb_training_data["Passing TDs per game"] = qb_training_data["Touchdowns"] / qb_training_data["GamesPlayed"]
qb_training_data["Rushing TDs per game"] = qb_training_data["RushTouchdowns"] / qb_training_data["GamesPlayed"]
qb_training_data = qb_training_data.drop(columns = ["Touchdowns", "RushTouchdowns", "GamesPlayed"])
qb_training_data

,Age,Completions,Attempts,Interceptions,College,Conference,Heisman,Passing Yard per attempt,Rushing Yard per attempt,Passing TDs per game,Rushing TDs per game
0,23,600,897,28,Florida St.,Atlantic Coast,0,8.629877,2.775168,1.093023,0.255814
1,22,988,1465,21,West Virginia,Big 12,0,7.960410,1.395918,2.227273,0.090909
2,23,646,1069,31,North Carolina St.,Atlantic Coast,0,6.932647,-2.531532,1.750000,0.083333
3,23,1001,1562,48,USC,Pac-12,0,7.891805,-0.856061,2.468085,0.127660
4,23,791,1312,28,Syracuse,Atlantic Coast,0,7.004573,0.694215,1.489362,0.106383
...,...,...,...,...,...,...,...,...,...,...,...
123,23,357,651,24,Virginia,Atlantic Coast,0,7.861751,2.748744,0.970588,0.411765
124,23,244,480,13,Kansas St.,Big 12,0,9.168750,4.055556,1.565217,1.000000
125,23,281,497,15,Miami (FL),Atlantic Coast,0,7.770624,-3.062500,0.742857,0.057143
126,22,863,1381,33,Tennessee,Southeastern,0,8.110789,-1.183007,1.977778,0.266667


Now we have completions and interceptions. I'm thinking we can have completion rate and interceptions per attempt?

In [17]:
qb_training_data["Completion Rate"] = qb_training_data["Completions"] / qb_training_data["Attempts"]
qb_training_data["Interception Rate"] = qb_training_data["Interceptions"] / qb_training_data["Attempts"]
qb_training_data = qb_training_data.drop(columns = ["Interceptions", "Completions", "Attempts"])
qb_training_data

,Age,College,Conference,Heisman,Passing Yard per attempt,Rushing Yard per attempt,Passing TDs per game,Rushing TDs per game,Completion Rate,Interception Rate
0,23,Florida St.,Atlantic Coast,0,8.629877,2.775168,1.093023,0.255814,0.668896,0.031215
1,22,West Virginia,Big 12,0,7.960410,1.395918,2.227273,0.090909,0.674403,0.014334
2,23,North Carolina St.,Atlantic Coast,0,6.932647,-2.531532,1.750000,0.083333,0.604303,0.028999
3,23,USC,Pac-12,0,7.891805,-0.856061,2.468085,0.127660,0.640845,0.030730
4,23,Syracuse,Atlantic Coast,0,7.004573,0.694215,1.489362,0.106383,0.602896,0.021341
...,...,...,...,...,...,...,...,...,...,...
123,23,Virginia,Atlantic Coast,0,7.861751,2.748744,0.970588,0.411765,0.548387,0.036866
124,23,Kansas St.,Big 12,0,9.168750,4.055556,1.565217,1.000000,0.508333,0.027083
125,23,Miami (FL),Atlantic Coast,0,7.770624,-3.062500,0.742857,0.057143,0.565392,0.030181
126,22,Tennessee,Southeastern,0,8.110789,-1.183007,1.977778,0.266667,0.624909,0.023896


Lmk if this assumption is wrong but I think the college doesn't really play a big role, but rather the conference as the stats mainly depend on your competion. So imma j drop the college.

In [18]:
qb_training_data = qb_training_data.drop(columns = ["College"])
qb_training_data

,Age,Conference,Heisman,Passing Yard per attempt,Rushing Yard per attempt,Passing TDs per game,Rushing TDs per game,Completion Rate,Interception Rate
0,23,Atlantic Coast,0,8.629877,2.775168,1.093023,0.255814,0.668896,0.031215
1,22,Big 12,0,7.960410,1.395918,2.227273,0.090909,0.674403,0.014334
2,23,Atlantic Coast,0,6.932647,-2.531532,1.750000,0.083333,0.604303,0.028999
3,23,Pac-12,0,7.891805,-0.856061,2.468085,0.127660,0.640845,0.030730
4,23,Atlantic Coast,0,7.004573,0.694215,1.489362,0.106383,0.602896,0.021341
...,...,...,...,...,...,...,...,...,...
123,23,Atlantic Coast,0,7.861751,2.748744,0.970588,0.411765,0.548387,0.036866
124,23,Big 12,0,9.168750,4.055556,1.565217,1.000000,0.508333,0.027083
125,23,Atlantic Coast,0,7.770624,-3.062500,0.742857,0.057143,0.565392,0.030181
126,22,Southeastern,0,8.110789,-1.183007,1.977778,0.266667,0.624909,0.023896
